# 推薦システムにおける評価手法の話

## 概要

この記事では、推薦システムにおける評価手法について詳述する。評価手法の定義や性質、応用例について数式とPythonのコードを用いて具体例を示す。また、評価手法のメリットとデメリットについても論じ、具体的な利用例として「movielens-100k」データセットを用いた実装例を紹介する。



## ソースコード


### github
- jupyter notebook形式のファイルは[こちら](https://github.com/hiroshi0530/wa-src/blob/master/rec/gr/06/06_nb.ipynb)

### google colaboratory
- google colaboratory で実行する場合は[こちら](https://colab.research.google.com/github/hiroshi0530/wa-src/blob/master/rec/gr/06/06_nb.ipynb)


## 実行環境
OSはmacOSである。LinuxやUnixのコマンドとはオプションが異なりますので注意していただきたい。

In [1]:
!sw_vers

ProductName:		macOS
ProductVersion:		13.5.1
BuildVersion:		22G90


In [2]:
!python -V

Python 3.9.17


pandasのテーブルを見やすいようにHTMLのテーブルにCSSの設定を行います。

In [ ]:
from IPython.core.display import HTML

style = """
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

    .dataframe thead th {
        text-align: left;
        padding: 5px;
    }

    .dataframe tbody tr th {
        vertical-align: top;
        padding: 5px;
    }

    .dataframe tbody tr:hover {
        background-color: #ffff99;
    }

    .dataframe {
        background-color: white;
        color: black;
        font-size: 16px;
    }

</style>
"""
HTML(style)

基本的なライブラリをインポートし watermark を利用してそのバージョンを確認しておきます。
ついでに乱数のseedの設定をします。

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
import random

import scipy
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

seed = 123
random_state = 123

random.seed(seed)
np.random.seed(seed)


from watermark import watermark

print(watermark(python=True, watermark=True, iversions=True, globals_=globals()))

Python implementation: CPython
Python version       : 3.9.17
IPython version      : 8.17.2

scipy     : 1.11.2
numpy     : 1.25.2
matplotlib: 3.8.1

Watermark: 2.4.3



In [ ]:
# 推薦システムにおける評価手法

## 概要
推薦システムは、ユーザーにパーソナライズされたコンテンツを提供するための重要な技術である。その性能を正確に評価することは、システムの改善とユーザー満足度の向上に直結する。本記事では、推薦システムの評価手法について、基本的な評価指標から特有の評価指標、オフライン評価とオンライン評価、そしてユーザー体験の評価までを網羅的に解説する。数式とPythonのコード例を交えて、具体的な評価手法を理解するための実践的なガイドを提供する。

## 1. はじめに
推薦システムは、オンラインショッピング、動画配信サービス、音楽ストリーミングなど多くの分野で利用されている。ユーザーに適切なアイテムを提供することで、エンゲージメントを高め、ビジネスの成功に寄与する。評価手法の選定と実施は、システムの性能を理解し、最適化するために不可欠である。以下にその具体的な評価手法を解説する。

## 2. 推薦システムの基本評価指標
推薦システムの性能を評価する基本的な指標には、精度評価とエラー率評価がある。

### 2.1 精度評価
推薦システムの精度を評価する基本指標として、正解率(Precision)、再現率(Recall)、およびF1スコアがある。これらの指標は、分類問題の評価においても広く用いられる。

- **正解率(Precision)**
  $$
  \text{Precision} = \frac{TP}{TP + FP}
  $$
  ここで、$TP$は真陽性(True Positives)、$FP$は偽陽性(False Positives)である。

  ```python
  from sklearn.metrics import precision_score

  y_true_list = [1, 0, 1, 1, 0, 1, 0, 0, 1, 1]
  y_pred_list = [1, 0, 1, 0, 0, 1, 0, 1, 1, 1]

  precision = precision_score(y_true_list, y_pred_list)
  print("Precision:", round(precision, 2))
  ```

- **再現率(Recall)**
  $$
  \text{Recall} = \frac{TP}{TP + FN}
  $$
  ここで、$FN$は偽陰性(False Negatives)である。

  ```python
  from sklearn.metrics import recall_score

  recall = recall_score(y_true_list, y_pred_list)
  print("Recall:", round(recall, 2))
  ```

- **F1スコア**
  $$
  \text{F1 Score} = \frac{2 \cdot \text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}
  $$

  ```python
  from sklearn.metrics import f1_score

  f1 = f1_score(y_true_list, y_pred_list)
  print("F1 Score:", round(f1, 2))
  ```

### 2.2 エラー率評価
エラー率を評価する指標として、平均絶対誤差(MAE)と二乗平均平方根誤差(RMSE)がある。これらの指標は、レーティング予測などの回帰問題においても重要である。

- **平均絶対誤差(MAE)**
  $$
  \text{MAE} = \frac{1}{N} \sum_{i=1}^{N} \left| y_i - \hat{y}_i \right|
  $$
  ここで、$N$はサンプル数、$y_i$は実際の値、$\hat{y}_i$は予測値である。

  ```python
  from sklearn.metrics import mean_absolute_error

  y_true_list = [3.5, 2.0, 4.0, 3.0, 5.0]
  y_pred_list = [3.7, 2.1, 3.9, 3.2, 4.8]

  mae = mean_absolute_error(y_true_list, y_pred_list)
  print("MAE:", round(mae, 2))
  ```

- **二乗平均平方根誤差(RMSE)**
  $$
  \text{RMSE} = \sqrt{\frac{1}{N} \sum_{i=1}^{N} \left( y_i - \hat{y}_i \right)^2}
  $$

  ```python
  from sklearn.metrics import mean_squared_error
  import numpy as np

  mse = mean_squared_error(y_true_list, y_pred_list)
  rmse = np.sqrt(mse)
  print("RMSE:", round(rmse, 2))
  ```

## 3. 推薦システム特有の評価指標
推薦システムには、特有の評価指標がいくつか存在する。これらは、一般的な精度評価やエラー率評価とは異なり、ユーザーの行動やランキングの精度を重視するものである。

### 3.1 ヒット率(Hit Rate)
ヒット率は、ユーザーが興味を示したアイテムが推薦リストに含まれている割合を示す。これは、推薦システムの基本的な成功指標の一つである。

- **ヒット率の定義**
  $$
  \text{Hit Rate} = \frac{\text{Number of Hits}}{\text{Total Number of Users}}
  $$

  ```python
  def hit_rate(recommended_list, relevant_list):
      hits = sum(1 for rec, rel in zip(recommended_list, relevant_list) if rec in rel)
      return hits / len(recommended_list)

  recommended_list = [1, 2, 3, 4, 5]
  relevant_list = [1, 2, 3, 6, 7]

  hr = hit_rate(recommended_list, relevant_list)
  print("Hit Rate:", round(hr, 2))
  ```

### 3.2 平均順位平均精度(MAP)
平均順位平均精度(MAP)は、推薦結果の順位情報を考慮した評価指標である。ユーザーにとっての有用性をより正確に評価できる。

- **MAPの定義**
  $$
  \text{MAP} = \frac{1}{|U|} \sum_{u \in U} \text{AP}(u)
  $$
  ここで、$\text{AP}(u)$はユーザー$u$の平均精度、$|U|$はユーザー数である。

  ```python
  def average_precision(recommended_list, relevant_list):
      hits = 0
      sum_precisions = 0
      for i, rec in enumerate(recommended_list):
          if rec in relevant_list:
              hits += 1
              sum_precisions += hits / (i + 1)
      return sum_precisions / len(relevant_list)

  recommended_list = [1, 2, 3, 4, 5]
  relevant_list = [1, 2, 3]

  ap = average_precision(recommended_list, relevant_list)
  print("Average Precision:", round(ap, 2))
  ```

### 3.3 正規化割引累積利得(NDCG)
正規化割引累積利得(NDCG)は、順位の重要性を考慮した評価指標である。高順位のアイテムがより重要視される。

- **NDCGの定義**
  $$
  \text{NDCG} = \frac{DCG}{IDCG}
  $$
  ここで、$DCG$は割引累積利得、$IDCG$は理想的な累積利得である。

  ```python
  def dcg(recommended_list, relevant_list):
      return sum((1 if rec in relevant_list else 0) / np.log2(idx + 2) for idx, rec in enumerate(recommended_list))

  def ndcg(recommended_list, relevant_list):
      dcg_val = dcg(recommended_list, relevant_list)
      idcg_val = dcg(sorted(relevant_list, reverse=True), relevant_list)
      return dcg_val / idcg_val

  recommended_list = [1, 2, 3, 4, 5]
  relevant_list = [1, 2, 3]

  ndcg_val = ndcg(recommended_list, relevant_list)
  print("NDCG:", round(ndcg_val, 2))
  ```

## 4. オフライン評価とオンライン評価
推薦システムの評価には、オフライン評価とオンライン評価の2種類がある。各手法にはそれぞれメリットとデメリ

ットが存在する。

### 4.1 オフライン評価
オフライン評価は、事前に収集したデータを用いて評価を行う手法である。

- **メリットとデメリット**
  メリットとして、低コストで迅速に評価が可能である。デメリットとして、実際のユーザー行動とは異なる可能性がある。

- **適用方法と事例**
  過去のデータを用いてシミュレーションを行い、システムの性能を評価する。例えば、MovieLensデータセットを使用して、評価を行うことができる。

  ```python
  import pandas as pd
  from sklearn.model_selection import train_test_split

  # データの読み込み
  ratings = pd.read_csv('https://files.grouplens.org/datasets/movielens/ml-100k/u.data', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])

  # 訓練データとテストデータに分割
  train_data, test_data = train_test_split(ratings, test_size=0.2)

  # 訓練データのサンプル表示
  print(train_data.head())
  ```

### 4.2 オンライン評価
オンライン評価は、実際のユーザーを対象に評価を行う手法である。

- **A/Bテスト**
  A/Bテストは、異なるバージョンのシステムを比較するために用いられる。ユーザーをランダムにグループに分け、それぞれに異なるバージョンを提供し、その効果を比較する。

- **メリットとデメリット**
  メリットとして、実際のユーザー行動を反映した評価が可能である。デメリットとして、実施に時間とコストがかかる。

  ```python
  # A/Bテストのシミュレーション例
  import numpy as np

  # ユーザーのクリック率を仮定
  group_a_clicks = np.random.binomial(1, 0.1, 1000)  # グループAのクリック率10%
  group_b_clicks = np.random.binomial(1, 0.15, 1000)  # グループBのクリック率15%

  # クリック率の平均を計算
  click_rate_a = np.mean(group_a_clicks)
  click_rate_b = np.mean(group_b_clicks)

  print("Group A Click Rate:", round(click_rate_a, 2))
  print("Group B Click Rate:", round(click_rate_b, 2))
  ```

## 5. ユーザー体験の評価
推薦システムの成功には、ユーザー体験の評価も重要である。これには、ユーザー満足度とエンゲージメントの評価が含まれる。

### 5.1 ユーザー満足度
ユーザー満足度は、アンケートやフィードバックを通じて評価される。これは、システムの改善に直接役立つ情報を提供する。

- **アンケートやフィードバックの活用**
  アンケートを通じてユーザーから直接意見を収集し、その結果をもとにシステムの改良を行う。

  ```python
  import pandas as pd

  # サンプルアンケートデータ
  feedback_data = {
      'user_id': [1, 2, 3, 4, 5],
      'satisfaction': [5, 4, 3, 4, 5]
  }

  feedback_df = pd.DataFrame(feedback_data)
  average_satisfaction = feedback_df['satisfaction'].mean()
  print("Average User Satisfaction:", round(average_satisfaction, 2))
  ```

### 5.2 エンゲージメント
エンゲージメント指標は、ユーザーがシステムをどれだけ頻繁に利用しているかを示す。これにより、ユーザーの忠実度を測ることができる。

- **エンゲージメント指標の定義と重要性**
  エンゲージメント指標は、ユーザーのシステム利用頻度や利用時間を測定する。これにより、ユーザーがシステムにどれだけ依存しているかを評価できる。

  ```python
  import pandas as pd

  # サンプルエンゲージメントデータ
  engagement_data = {
      'user_id': [1, 2, 3, 4, 5],
      'sessions': [10, 15, 5, 20, 25],
      'time_spent': [300, 450, 150, 600, 750]  # in minutes
  }

  engagement_df = pd.DataFrame(engagement_data)
  average_sessions = engagement_df['sessions'].mean()
  average_time_spent = engagement_df['time_spent'].mean()
  print("Average Sessions per User:", round(average_sessions, 2))
  print("Average Time Spent per User (minutes):", round(average_time_spent, 2))
  ```

## 6. まとめ
評価手法の選択と組み合わせが重要である。継続的な評価と改善を行い、システムの性能を最適化することが求められる。評価は、システムの成功とユーザー満足度の向上に不可欠である。

## 結論
この記事では、推薦システムの評価手法について、基本指標から特有の指標、オフライン評価とオンライン評価、ユーザー体験の評価まで幅広く解説した。適切な評価手法を選び、継続的にシステムの改善を行うことが、成功する推薦システムの構築には不可欠である。

参考文献：
1. Ricci, F., Rokach, L., & Shapira, B. (2011). Introduction to Recommender Systems Handbook. Springer.
2. Schafer, J. B., Konstan, J., & Riedl, J. (1999). Recommender Systems in E-Commerce. Proceedings of the 1st ACM conference on Electronic commerce.
3. Aggarwal, C. C. (2016). Recommender Systems: The Textbook. Springer.
4. Koren, Y., Bell, R., & Volinsky, C. (2009). Matrix Factorization Techniques for Recommender Systems. Computer, 42(8), 30-37.


## 評価手法の定義と性質

推薦システムの評価手法は、システムがユーザーに対してどれだけ適切な推薦を行っているかを測定する方法である。評価手法は、推薦システムの性能を比較・改善するために不可欠である。

### 主要な評価指標

推薦システムの評価には、以下の主要な指標が使用される。

1. **精度 (Precision)**:
   推薦されたアイテムのうち、ユーザーが実際に好むアイテムの割合。
   $$ \text{Precision} = \frac{|\{推奨アイテム\} \cap \{好まれるアイテム\}|}{|\{推奨アイテム\}|} $$

2. **再現率 (Recall)**:
   ユーザーが好むアイテムのうち、推薦されたアイテムの割合。
   $$ \text{Recall} = \frac{|\{推奨アイテム\} \cap \{好まれるアイテム\}|}{|\{好まれるアイテム\}|} $$

3. **F1スコア (F1 Score)**:
   精度と再現率の調和平均。
   $$ \text{F1 Score} = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}} $$

4. **平均絶対誤差 (MAE)**:
   推薦システムの予測と実際の評価の絶対値の平均。
   $$ \text{MAE} = \frac{1}{N} \sum_{i=1}^{N} |r_i - \hat{r}_i| $$

5. **二乗平均平方根誤差 (RMSE)**:
   推薦システムの予測と実際の評価の二乗平均平方根。
   $$ \text{RMSE} = \sqrt{\frac{1}{N} \sum_{i=1}^{N} (r_i - \hat{r}_i)^2} $$

## 応用例と実装

次に、これらの評価指標を用いて「movielens-100k」データセットを使った実装例を示す。

### データセットの準備

まず、「movielens-100k」データセットをロードし、評価行列を準備する。



In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt

# データセットの読み込み
ratings = pd.read_csv("u.data", sep="\t", header=None, names=["user_id", "movie_id", "rating", "timestamp"])

# データセットの分割
train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)

# 評価行列の作成
R_train = train_data.pivot(index="user_id", columns="movie_id", values="rating").fillna(0)
R_test = test_data.pivot(index="user_id", columns="movie_id", values="rating").fillna(0)

# 評価行列をnumpy配列に変換
R_train = R_train.values
R_test = R_test.values

In [ ]:



### 評価指標の計算

次に、予測評価行列を生成し、評価指標を計算する。ここでは、簡単な平均ベースの予測を使用する。



In [ ]:
# ユーザー平均に基づく予測
user_mean = np.mean(R_train, axis=1)
R_pred = np.tile(user_mean, (R_train.shape[1], 1)).T

# MAEとRMSEの計算
mae = mean_absolute_error(R_test[R_test > 0], R_pred[R_test > 0])
rmse = sqrt(mean_squared_error(R_test[R_test > 0], R_pred[R_test > 0]))

print(f"MAE: {round(mae, 2)}")
print(f"RMSE: {round(rmse, 2)}")

In [ ]:


### 精度、再現率、F1スコアの計算

次に、精度、再現率、F1スコアを計算する。



In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# 二値予測: 予測評価が3以上である場合を「好まれる」とする
R_pred_binary = (R_pred >= 3).astype(int)
R_test_binary = (R_test >= 3).astype(int)

# 精度、再現率、F1スコアの計算
precision = precision_score(R_test_binary[R_test > 0], R_pred_binary[R_test > 0])
recall = recall_score(R_test_binary[R_test > 0], R_pred_binary[R_test > 0])
f1 = f1_score(R_test_binary[R_test > 0], R_pred_binary[R_test > 0])

print(f"Precision: {round(precision, 2)}")
print(f"Recall: {round(recall, 2)}")
print(f"F1 Score: {round(f1, 2)}")



## メリットとデメリット

### メリット

- **評価の多様性**: 複数の評価指標を使用することで、システムの性能を多角的に評価できる。
- **改善の指針**: 指標ごとにシステムの強みや弱点が明確になり、改善の方向性が見える。

### デメリット

- **計算コスト**: データが大規模になると、評価指標の計算に時間がかかる。
- **バイアス**: 特定の評価指標に偏ると、システムの総合的な性能が見落とされる可能性がある。

## 結論

この記事では、推薦システムにおける評価手法について詳述した。具体的な定義や数式、Pythonコードを用いた具体例を示し、メリットとデメリットを論じた。評価手法は、システムの性能を多角的に評価し、改善の方向性を見出すために重要である。

### 参考文献

- Wikipedia: [Precision and recall](https://en.wikipedia.org/wiki/Precision_and_recall)
- Scikit-learn: [Metrics and scoring](https://scikit-learn.org/stable/modules/model_evaluation.html)